In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR, MultiStepLR

In [ ]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
X = X/255
y = y.astype(int)

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
import cv2
from random import choice
def f1(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    return sobel_x, sobel_y
def f2(image):
  z = choice([1,2,3,4,5,-1,-2,-3,-4,-5])
  center = (14, 14)
  M = cv2.getRotationMatrix2D(center, 12*z, 1.0)
  return cv2.warpAffine(image, M, (28, 28))

In [ ]:
x = X_train.reshape(X_train.shape[0],28,28)
p = [f2(x[0])]
h = [y_train[0]]
ch=0
for i in range(3, 60000, 3):
  v = f2(x[i])
  v1 = y_train[i]
  p.append(v)
  h.append(v1)
  if ch%10000==0:
    print(ch, len(p), len(h))
  ch+=1
p = np.array(p)
h = np.array(h)
p.shape, h.shape

In [ ]:
def shuf(u1, u2):
  rm = np.arange(u1.shape[0])
  np.random.shuffle(rm)
  return (u1[rm], u2[rm])

In [ ]:
X_train = np.concat([X_train, p.reshape(p.shape[0], 784)], axis=0)
y_train = np.concat([y_train, h], axis=0)
X_train, y_train = shuf(X_train, y_train)
print(X_train.shape, y_train.shape)

In [ ]:
x = X_train.reshape(X_train.shape[0],28,28)
#x = f1(x)[0]
u=45
plt.subplot(1, 2, 1)
plt.imshow(x[u], cmap="Greys")
plt.subplot(1, 2, 2)
plt.imshow(f2(x[u]), cmap="Greys")


In [ ]:
plt.figure(figsize=(6,6))
for i in range(10):
  plt.subplot(1, 10, i+1)
  plt.imshow(X_train[i].reshape(28, 28), cmap="Greys")
plt.show()

In [ ]:
nn1 = 80
nn2 = 108
nn3 = 156
nn4 = 244
nn5 = 319
nn6 = 435
nn7 = 2000
nn8 = 500
nc = 10
lr = 0.0001
bs=64
dev = "cuda"
ep=50

In [ ]:
train_dset = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
train_loader =  torch.utils.data.DataLoader(train_dset, batch_size=bs)
test_dset = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())
test_loader =  torch.utils.data.DataLoader(test_dset, batch_size=bs)

In [ ]:
class FCN(nn.Module):
  def __init__(self):
    super(FCN, self).__init__()
    self.cc1 = nn.Conv2d(1, nn1, 5)
    self.bn1 = nn.BatchNorm2d(nn1)
    self.cc2 = nn.Conv2d(nn1, nn2, 5)
    self.bn2 = nn.BatchNorm2d(nn2)
    self.cc3 = nn.Conv2d(nn2, nn3, 5)
    self.bn3 = nn.BatchNorm2d(nn3)
    self.cc4 = nn.Conv2d(nn3, nn4, 5)
    self.bn4 = nn.BatchNorm2d(nn4)
    self.cc5 = nn.Conv2d(nn4, nn5, 3)
    self.bn5 = nn.BatchNorm2d(nn5)
    self.cc6 = nn.Conv2d(nn5, nn6, 3)
    self.bn6 = nn.BatchNorm2d(nn6)
    self.fc1 = nn.Linear(27840, nn7)
    self.bn7 = nn.BatchNorm1d(nn7)
    self.fc2 = nn.Linear(nn7, nn8)
    self.bn8 = nn.BatchNorm1d(nn8)
    self.fc3 = nn.Linear(nn8, nc)

  def forward(self, x):
    z1 = F.relu(self.bn1(self.cc1(x)))
    z2 = F.relu(self.bn2(self.cc2(z1)))
    z3 = F.relu(self.bn3(self.cc3(z2)))
    z4 = F.relu(self.bn4(self.cc4(z3)))
    z5 = F.relu(self.bn5(self.cc5(z4)))
    z6 = F.relu(self.bn6(self.cc6(z5)))
    #print(z6.shape)
    z6 = nn.Flatten()(z6)
    #print(z6.shape)
    z7 = F.dropout(F.relu(self.bn7(self.fc1(z6))), p=0.5)
    z8 = F.relu(self.bn8(self.fc2(z7)))
    z9 = self.fc3(z8)
    return z9

In [ ]:
net = FCN()
net.to(dev)

In [ ]:
crit = nn.CrossEntropyLoss()
opt = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = MultiStepLR(opt, milestones=[6, 15, 40], gamma=0.1)

In [ ]:
def vl():
  net.eval()
  with torch.no_grad():
    cur, tot = 0, 0
    for a, b in test_loader:
      a = a.to(dev)
      b = b.to(dev)
      a = a.reshape(a.shape[0], 1, 28, 28)
      out = net(a)
      _, pred = torch.max(out, dim=1)
      tot+=b.shape[0]
      cur+=(pred==b).sum().item()
    print('accuracy:', cur/tot)

In [ ]:
q = []
tot = len(train_loader)
for i in range(ep):
    net.train()
    for j, (a, b) in enumerate(train_loader):
      a = a.to(dev)
      b = b.to(dev)
      #print(a.shape)
      a = a.reshape(a.shape[0], 1, 28, 28)
      #print(a.shape, a[0])
      out = net(a)
      loss = crit(out, b)
      opt.zero_grad()
      loss.backward()
      opt.step()
      if(j+1)%200==0:
        q.append(loss.item())
        print('Epoch {}/{} Step {}/{} LossL{:.4f}'.format(i+1, ep, j+1, tot, loss.item()))
    scheduler.step()
    vl()
plt.scatter(range(len(q)), q, s=3, c='black')

NameError: name 'train_loader' is not defined

In [ ]:
net.eval()
with torch.no_grad():
  cur, tot = 0, 0
  for a, b in test_loader:
    a = a.to(dev)
    b = b.to(dev)
    a = a.reshape(a.shape[0], 1, 28, 28)
    out = net(a)
    _, pred = torch.max(out, dim=1)
    tot+=b.shape[0]
    cur+=(pred==b).sum().item()
  print('accuracy:', cur/tot)

In [ ]:
torch.save(net.state_dict(), 'n1.pth')

In [ ]:
from google.colab import files
files.download('/content/cfm1.pth')